# Stoneburner, Kurt
- ## DSC 540 - Week XX/XX
- ## Chapter X, Exercise X


### Wrangling the BoingBoing Candy Survey
This is fun. Boing Boing has been in my daily reading rotation for a longtime. So of course I have to work with the data. It doesn't hit the same notes as the symphony, but the data is sweet.

In [2]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

In [3]:
#There are three years of the Candy surveys, import each one
df_2015 = pd.read_excel("z_sup_wk07_08_candy2015.xlsx")
df_2016 = pd.read_excel("z_sup_wk07_08_candy2016.xlsx")
df_2017 = pd.read_excel("z_sup_wk07_08_candy2017.xlsx")


In [4]:
#//*** Rename some columns to help with Fuzzy Matching
df_2015.rename(columns = {" [Box’o’ Raisins]" : " [Box'o'Raisins]", " [Dark Chocolate Hershey]" : " [Hershey's Dark Chocolate]"," [Hershey’s Kissables]" : " [Hershey's Kisses]"," [Licorice]" :" [Licorice (not black)]"}, inplace=True)


print(list( df_2015.columns [ df_2015.columns.str.match('^.\[')]))

#//*** It's a little early for Merging, But let's look at the columns to determine which ones have candy types.
#//*** No sense in cleaning columns we are not going to use.
#//*** 2015 - Candy names start at the beginning of the column name and are wrapped in brackets

#//*** Get Candy Columns for 2015. Use Regex to to match the start of the string, any character then Bracket.
#//*** This eliminates the degrees of separation quetions.
#/**** Trim the leading space and brackets while we are at it. 
cols_2015 = list( df_2015.columns [ df_2015.columns.str.match('^.\[')])

#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2015_clean = list(df_2015.columns [ df_2015.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',""))
#//*** This is a good starting point for 2015 columns

print(f"{cols_2015_clean}")


[' [Butterfinger]', ' [100 Grand Bar]', ' [Anonymous brown globs that come in black and orange wrappers]', ' [Any full-sized candy bar]', ' [Black Jacks]', ' [Bonkers]', ' [Bottle Caps]', " [Box'o'Raisins]", ' [Brach products (not including candy corn)]', ' [Bubble Gum]', ' [Cadbury Creme Eggs]', ' [Candy Corn]', ' [Vials of pure high fructose corn syrup, for main-lining into your vein]', ' [Candy that is clearly just the stuff given out for free at restaurants]', ' [Cash, or other forms of legal tender]', ' [Chiclets]', ' [Caramellos]', ' [Snickers]', " [Hershey's Dark Chocolate]", ' [Dental paraphenalia]', ' [Dots]', ' [Fuzzy Peaches]', ' [Generic Brand Acetaminophen]', ' [Glow sticks]', ' [Broken glow stick]', ' [Goo Goo Clusters]', " [Good N' Plenty]", ' [Gum from baseball cards]', ' [Gummy Bears straight up]', ' [Creepy Religious comics/Chick Tracts]', ' [Healthy Fruit]', ' [Heath Bar]', " [Hershey's Kisses]", ' [Hershey’s Milk Chocolate]', ' [Hugs (actual physical hugs)]', ' [Jol

In [9]:
#//*** Get Candy Columns for 2016. These questions are structured using the same methods as 2015.
cols_2016 = list( df_2016.columns [ df_2016.columns.str.match('^.\[')])
#.str.replace('^.',"").str.replace('[',"").str.replace(']',"")

print(cols_2016)
#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2016_clean = list(df_2016.columns [ df_2016.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',""))

print(cols_2016_clean)

[' [100 Grand Bar]', ' [Anonymous brown globs that come in black and orange wrappers]', ' [Any full-sized candy bar]', ' [Black Jacks]', ' [Bonkers (the candy)]', ' [Bonkers (the board game)]', ' [Bottle Caps]', " [Box'o'Raisins]", ' [Broken glow stick]', ' [Butterfinger]', ' [Cadbury Creme Eggs]', ' [Candy Corn]', ' [Candy that is clearly just the stuff given out for free at restaurants]', ' [Caramellos]', ' [Cash, or other forms of legal tender]', ' [Chardonnay]', ' [Chick-o-Sticks (we don’t know what that is)]', ' [Chiclets]', ' [Coffee Crisp]', ' [Creepy Religious comics/Chick Tracts]', ' [Dental paraphenalia]', ' [Dots]', ' [Dove Bars]', ' [Fuzzy Peaches]', ' [Generic Brand Acetaminophen]', ' [Glow sticks]', ' [Goo Goo Clusters]', " [Good N' Plenty]", ' [Gum from baseball cards]', ' [Gummy Bears straight up]', ' [Hard Candy]', ' [Healthy Fruit]', ' [Heath Bar]', " [Hershey's Dark Chocolate]", ' [Hershey’s Milk Chocolate]', " [Hershey's Kisses]", ' [Hugs (actual physical hugs)]', '

In [10]:
#//*** Get Candy Columns for 2017. All Candy questions Begin with Q6 in the Column Name.
cols_2017 = list(df_2017.columns [ df_2017.columns.str.match('^Q6')])
#.str.replace('^Q6 \| ','')
cols_2017_clean = list(df_2017.columns [ df_2017.columns.str.match('^Q6')].str.replace('^Q6 \| ',''))
print(cols_2017)
print(cols_2017_clean)

['Q6 | 100 Grand Bar', 'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)', 'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks', 'Q6 | Bonkers (the candy)', 'Q6 | Bonkers (the board game)', 'Q6 | Bottle Caps', "Q6 | Box'o'Raisins", 'Q6 | Broken glow stick', 'Q6 | Butterfinger', 'Q6 | Cadbury Creme Eggs', 'Q6 | Candy Corn', 'Q6 | Candy that is clearly just the stuff given out for free at restaurants', 'Q6 | Caramellos', 'Q6 | Cash, or other forms of legal tender', 'Q6 | Chardonnay', 'Q6 | Chick-o-Sticks (we don’t know what that is)', 'Q6 | Chiclets', 'Q6 | Coffee Crisp', 'Q6 | Creepy Religious comics/Chick Tracts', 'Q6 | Dental paraphenalia', 'Q6 | Dots', 'Q6 | Dove Bars', 'Q6 | Fuzzy Peaches', 'Q6 | Generic Brand Acetaminophen', 'Q6 | Glow sticks', 'Q6 | Goo Goo Clusters', "Q6 | Good N' Plenty", 'Q6 | Gum from baseball cards', 'Q6 | Gummy Bears straight up', 'Q6 | Hard Candy', 'Q6 | Healthy Fruit', 'Q6 | Heath Bar', "Q6 | Hershey's Dark Chocolate", 

## Chapter 8 - Task1 ###
**Merge the Dataframes**

Merging the dataframes first means all the cleaning can be applied once.

1. Combine the dates
2. Add a year column
3. Use FuzzyMatching on the the clean column names to find the common columns in all three datasets

In [7]:
#//*******************************************************************************************************
#//*** Combine 2015, 2016, & 2017 datasets
#//*** Build a year column indicating which year generated the results
#//*** Use Fuzzy Matching on the column name to find the common columns between all three dataframes
#//*******************************************************************************************************

#//*** Initialize the dataframe
candy_df = pd.DataFrame()

print(df_2015[df_2015.columns[0]])
print(df_2016[df_2016.columns[0]])
print(df_2017[df_2017.columns[0]])

#//***********************************************************************************************
#//*** Generate a year value for each value
#//*** Using 3 generators feels stupid And pythonic. But why not? it works.
#//*** Each generator builds a list the length of each data frame with a year value.
#//*** These lists are added together (concatenated), shoved into a Series, and added to candy_df
#//***********************************************************************************************
candy_df['year'] = pd.Series( 
    [2015 for x in range(0,len(df_2015))] + 
    [2016 for x in range(0,len(df_2016))] + 
    [2017 for x in range(0,len(df_2017))])                 






0      2015-10-23 08:46:20.451
1      2015-10-23 08:46:51.583
2      2015-10-23 08:47:34.285
3      2015-10-23 08:47:58.964
4      2015-10-23 08:48:11.719
                 ...          
5625   2015-10-31 05:23:40.526
5626   2015-10-31 05:29:26.937
5627   2015-10-31 06:13:29.083
5628   2015-10-31 06:26:52.566
5629   2015-10-31 06:41:31.904
Name: Timestamp, Length: 5630, dtype: datetime64[ns]
0      2016-10-24 05:09:23.033
1      2016-10-24 05:09:54.798
2      2016-10-24 05:13:06.734
3      2016-10-24 05:14:17.192
4      2016-10-24 05:14:24.625
                 ...          
1254   2016-10-29 16:53:52.516
1255   2016-10-30 06:53:54.735
1256   2016-10-30 11:06:10.827
1257   2016-10-30 16:07:26.539
1258   2016-10-30 17:06:45.660
Name: Timestamp, Length: 1259, dtype: datetime64[ns]
0       90258773
1       90272821
2       90272829
3       90272840
4       90272841
          ...   
2455    90314359
2456    90314580
2457    90314634
2458    90314658
2459    90314802
Name: Internal ID, Length

In [8]:
#//*****************************************************************
#//*** Fuzzy match for columns that exist in all three dataframes
#//*****************************************************************
#//*** 2015 has the fewest columns, Use fuzzy matching to compare 2015 with 2016 then 2017 column names
#//*** This keeps values that only match a score of 100, the rest get tossed aside.A record of skipped columns
#//*** is kept in cols_to_drop for reference. 
#//*** Fuzzy Matching caught all but 4 (technically 5) columns. I manually renamed these columns earlier in the code.
#//*** I did drop JoyJoy Mit Iodine, becuase it a Simpson's treehouse of Horrors Gag. Maybe it should be left in.
#//*** But I contend "The Simpsons Did It"

#//*** Loop through the index values of 2015
#//*** The index is used to link the cleaned name to the original column name.
for index_2015 in range(0, len(cols_2015_clean)):
    
    #//*** Get the 2015 column name
    value_2015 = cols_2015_clean[index_2015]
    
    #//*** Initialize the 2016 best values
    best_2016_score = -1
    best_2016_index = -1
    best_2016_value = ""

    #//*** Loop through 2016 Column Names and Fuzzy Match with the current 2015 column name
    #//*** Get the highest scoring value
    for index_2016 in range(0,len(cols_2016_clean)):
        
        #//*** Get a 2016 column name
        value_2016 = cols_2016_clean[index_2016]
        
        #//*** Get a Fuzzy Score comparing 2015 with 2016
        score = fuzz.ratio(value_2015,value_2016)

        #//*** Update the best score if applicable
        if score > best_2016_score:
            best_2016_score = score
            best_2016_index = index_2016
            best_2016_value = value_2016


    #//*** Initialize the 2017 best values
    best_2017_score = -1
    best_2017_index = -1
    best_2017_value = ""

    #//*** Loop through 2017 Column Names and Fuzzy Match with the current 2015 column name
    #//*** Get the highest scoring value
    for index_2017 in range(0,len(cols_2017_clean)):

        #//*** Get a 2017 column name
        value_2017 = cols_2017_clean[index_2017]
        
        #//*** Get a Fuzzy Score comparing 2015 with 2017
        score = fuzz.ratio(value_2015,value_2017)

        #//*** Update the best score if applicable
        if score > best_2017_score:
            best_2017_score = score
            best_2017_index = index_2017
            best_2017_value = value_2017
    
    #//*** Keep a list of unable to match columns, These should all be fake/non-sensical/non-matching
    #//*** If it didn't match 2016, then it isn't a match
    if best_2016_score < 100:
        cols_to_drop.append(f"{value_2015} - {best_2016_value} - {best_2017_value}")
    else:
        #//*********************************************************************************************************
        #//*** It's a match!!! 
        #//*** Assemble the common columns and concatenate the series, then add to the final assembled dataframe
        #//*** This is a totally awkward janky method to merge. But it works
        #//*** This is similar to the year generator earlier, except this time we're grabbing
        #//*** Columns based on index value to generate a Series, which is converted into lists for 2015,2016,2017
        #//*** The lists are added (concatenated), converted back into a Series and shoved into the dataframe
        #//*** Like I said, Janky...but effective.
        #//*********************************************************************************************************
        candy_df[value_2015] = pd.Series(list(df_2015[cols_2015[index_2015]]) 
                                         + list(df_2016[cols_2016[index_2016]])
                                         + list(df_2017[cols_2017[index_2017]])
                                        )

print(candy_df.head(5))
        

NameError: name 'cols_to_drop' is not defined

## Chapter 7 - Task1 ###
**Filter Out Missing Values**

In the Drop rows where no Candy values were filled out

In [ ]:
#//*** Drop rows where None of the Candy columns filled out
#//*** Every Year has a value. Use a subset of columns without the year
#//*** Technically we could leave these in, and replace the values with zeros
#//*** Since they have no value "GIT 'em Out of here!!!'"
print(f"2015 length  before cleaning: {len(candy_df)}")
candy_df.dropna(axis=0,how='all',subset=candy_df.columns[1:],inplace=True)

print(f"2015 length after cleaning: {len(candy_df)}")

## Chapter 7 - Task 2###
**Replace Values**

In many cases usere did not fill out all the form data. If we dropped these values, there wouldn't be much of a survey. Replace NaN with 0. Which will be used as a categorical for not answered.

In [ ]:
#//*** Replace NaN with 0 in 2015 Data
df_2015.fillna(value=0,axis=0,inplace=True)
print(df_2015.head(10))

#//*** Replace NaN with 0 in 2016 Data
df_2016.fillna(value=0,axis=0,inplace=True)

#//*** Replace NaN with 0 in 2017 Data
df_2017.fillna(value=0,axis=0,inplace=True)

In [ ]:
print(df_2015)

## Chapter 7 - Task 3###
**Transform Data**

Convert the categorical descriptions from Despair and Joy to 1 and 2

In [ ]:
#//*** Check for unique values in the candy columsn
#//*** Looking for miscodes or irregularities
print(pd.unique(df_2015[cols_2015].values.ravel()))
print(pd.unique(df_2016[list(cols_2016)].values.ravel()))
print(pd.unique(df_2017[list(cols_2017)].values.ravel()))


#for x in cols_2016:
#    print(df_2016[x].iloc[0])

In [ ]:
#//*** Apply a map to e
def remap_df(input_df, input_cols, input_map):
    
    for col in input_cols:
        
        input_df[col] = input_df[col].map(input_map, na_action='ignore')
    

#//**************************************************************************
#//*** Create Map based on Unique Values
#//*** Replace Joy with 1, because joy should always go first
#//*** Replace Despair with 2, because despair is greater than joy
#//*** Replace Meh with 3, because apathy conquers all
#//**************************************************************************

remap_ratings = {'JOY' : 1, 'DESPAIR' : 2, 'MEH' : 3, 0 : 0}

#//*** Mapping only applies to a series.
#//*** Loop through the dataframes and apply the mapping to each column.
#//*** This feels like a good use for a generator or Lambda expression.
#//*** But I'm not that cool today.
#//*** Do it with Loops because ... Loops

#//*** Remaps can only be applied once. Adding a conditional to run if JOY exist as a unique value
#//*** This speeds up coding / debugging due to the reusable nature of iPython
#// Remap 2015
if 'JOY' in list(pd.unique(df_2015[cols_2015].values.ravel())):
    remap_df(df_2015, cols_2015, remap_ratings)

#// Remap 2016
if 'JOY' in list(pd.unique(df_2016[cols_2016].values.ravel())):
    remap_df(df_2016, cols_2016, remap_ratings)

#// Remap 2017    
if 'JOY' in list(pd.unique(df_2017[cols_2017].values.ravel())):
    remap_df(df_2017, cols_2017, remap_ratings)


    
#//*** Visually Check our Work   
print(df_2015.head(5))
print(df_2016.head(5))
print(df_2017.head(5))
